In [24]:
%pip install "git+https://github.com/jemisjoky/TorchMPS.git"
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
%pip install torchmetrics
from torchmetrics.classification import MulticlassAccuracy
from torchmps import MPS
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/jemisjoky/TorchMPS.git to /tmp/pip-req-build-8i12k4bh
  Running command git clone --filter=blob:none --quiet https://github.com/jemisjoky/TorchMPS.git /tmp/pip-req-build-8i12k4bh
  Resolved https://github.com/jemisjoky/TorchMPS.git to commit 6c0bc1a8e2c15acba8570ca9ffe2b4a0c7135165
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Hyperparameters

In [25]:
# FC to 2-MPS

# Hardware hyperparameters
chosen_device = torch.device('cuda' 
if torch.cuda.is_available() else 'cpu')

# Data hyperparameters
nb_train_HP = 2000
nb_test_HP = 500
batch_sz_HP = 150
batch_sz_HP = min(batch_sz_HP, nb_train_HP)
nb_classes_HP = 10

# Teacher hyperparameters
nepochs_teacher_HP = 25
teacher_loss_HP = nn.CrossEntropyLoss()
teacher_lr_HP = 1e-2
teacher_reg_HP = 0.01
teacher_hidden_size_HP = 70
# Student hyperparameters
# MPS parameters
bond_dim_HP = 20
adaptive_mode_HP = False
periodic_bc_HP = False
use_cust_feature_map_HP = True

# Training parameters
nepochs_student_HP = 25 
student_lr_HP = 1e-4
student_reg_HP = 0.01
student_loss_HP = nn.KLDivLoss(reduction = "batchmean", log_target = True)

# Gaussian parameters
gauss_epochs_HP = 15 # number of epochs with added gaussian noise
gn_var_HP = 0.3 #added gaussian noise variance
gn_mean_HP = 0 #added gaussian noise mean
#nepochs_student_HP = 25 + gauss_epochs_HP


# Premilinaries: Importing the data and utils subroutines

In [26]:
# Import the mnist train dataset
train_set = torchvision.datasets.MNIST(
    root = './datasets', train = True,   
    transform = transforms.ToTensor(),  download = True )

# Create a training batch iterator
train_subset = torch.utils.data.SubsetRandomSampler(range(nb_train_HP))
train_iterator = torch.utils.data.DataLoader(
    dataset = train_set, 
    sampler = train_subset, batch_size=batch_sz_HP
    )

# Import the mnist test set
test_set = torchvision.datasets.MNIST(
    root = './datasets',
    train = False, transform = transforms.ToTensor(),  download = True
    )
# Create a testing batch iterator
test_subset = torch.utils.data.SubsetRandomSampler(range(nb_test_HP))
test_iterator = torch.utils.data.DataLoader(
    dataset = test_set, 
    sampler = test_subset, batch_size = batch_sz_HP
    )

In [27]:
# Returns the validation set classification accuracy
# of the given input model (this is a higher order function)
def get_acc(model, iterator):
    # Get the validation set classification accuracy
    total_good_classifications = 0
    acc_metric = MulticlassAccuracy(num_classes=nb_classes_HP).to(chosen_device)
    for (x_mb, y_mb) in iterator:
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)
        # Add the number of datapoints we classified right to the total
        batch_size = x_mb.size()[0]
        y_hat = model(x_mb)
        batch_good_classifications = batch_size * acc_metric(y_hat, y_mb)
        total_good_classifications += batch_good_classifications
    return total_good_classifications / nb_test_HP # divide by total size

In [28]:
# Create the fcnn class
class FCNN(nn.Module):
    def __init__(self):
        super(FCNN, self).__init__()
        self.relu = nn.ReLU()
        self.lin1 = nn.Linear(784, teacher_hidden_size_HP)
        self.lin2 = nn.Linear(teacher_hidden_size_HP, teacher_hidden_size_HP)
        self.lin3 = nn.Linear(teacher_hidden_size_HP, teacher_hidden_size_HP)
        self.lin4 = nn.Linear(teacher_hidden_size_HP, teacher_hidden_size_HP)
        self.lin5 = nn.Linear(teacher_hidden_size_HP, teacher_hidden_size_HP)
        self.lin6 = nn.Linear(teacher_hidden_size_HP, 10)

    def middleforward(self, x):
        y = self.lin1(x)
        y = self.relu(y)
        y = self.lin2(y)
        y = self.relu(y)
        y = self.lin3(y)
        y = self.relu(y)
        return y

    def forward(self, x):
        y = self.lin1(x)
        y = self.relu(y)
        y = self.lin2(y)
        y = self.relu(y)
        y = self.lin3(y)
        y = self.relu(y)
        y = self.lin4(y)
        y = self.relu(y)
        y = self.lin5(y)
        y = self.relu(y)
        y = self.lin6(y)
        y = self.relu(y)
        return y

#Instantiate and put the model on the chosen device
teacher = FCNN().to(chosen_device)

#Instantiate the optimizer
teacher_optimizer = torch.optim.Adam(teacher.parameters())

# Create an array to store the val loss
# of the student at each epoch
teacher_test_loss = []
teacher_train_loss = []

# Training loop 
for epoch in range(nepochs_teacher_HP):
    for (x_mb, y_mb) in train_iterator:
        # Flatten the MNIST images, which come in matrix form
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)

        teacher_output = teacher(x_mb) 

        # Backpropagation
        loss = teacher_loss_HP(teacher_output, y_mb)
        loss.backward()
        teacher_optimizer.step()
        teacher_optimizer.zero_grad()

    #teacher_train_loss.append( round(get_acc(teacher, train_iterator).item(), 3) )
    teacher_test_loss.append( round(get_acc(teacher, test_iterator).item(), 5) )

print("Teacher results:")
print("Epochs: ", np.arange(1, nepochs_teacher_HP+1).tolist())
print("Train loss: ", teacher_train_loss)
print("Test loss: ", teacher_test_loss)


Teacher results:
Epochs:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
Train loss:  []
Test loss:  [0.18517, 0.26074, 0.36587, 0.42696, 0.49921, 0.4977, 0.51645, 0.52437, 0.52214, 0.51872, 0.51776, 0.51073, 0.52645, 0.53281, 0.54118, 0.53812, 0.5304, 0.53476, 0.53532, 0.53594, 0.55177, 0.53801, 0.53558, 0.54509, 0.55231]


# Training the student model

In [30]:
# Initialize the MPS modules
mps1 = MPS(
    input_dim = 28 ** 2,
    feature_dim = 2,
    output_dim = teacher_hidden_size_HP,
    bond_dim = bond_dim_HP
).to(chosen_device)

mps2 = MPS(
    input_dim = teacher_hidden_size_HP, 
    feature_dim = 2,
    output_dim = nb_classes_HP,
    bond_dim = bond_dim_HP
).to(chosen_device)

if use_cust_feature_map_HP: 
  feature_map_HP = lambda x : torch.tensor([1, x], device=chosen_device, requires_grad=False)
  mps1.register_feature_map(feature_map_HP)
  mps2.register_feature_map(feature_map_HP)

# Instantiate the optimizer and softmax
mps1_optimizer = torch.optim.Adam(
    mps1.parameters(), lr = student_lr_HP, weight_decay = student_reg_HP
)

mps2_optimizer = torch.optim.Adam(
    mps2.parameters(), lr = student_lr_HP, weight_decay = student_reg_HP
)


# Used on the inputs before the loss function
LogSoftmax = nn.LogSoftmax(dim=1)

# Create an array to store the val loss
# of the student at each epoch
stud_test_loss = []
stud_train_loss = []

def student(x):
  return mps2( mps1( x ) )

mse_loss = nn.MSELoss()

# Mps2 training loop 
for epoch in range(nepochs_student_HP * 2):
    for (x_mb, y_mb) in train_iterator:
        # Flatten the MNIST images, which come in matrix form
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)

        # Train mps1
        if (epoch < nepochs_student_HP):
          teacher_middle_output = teacher.middleforward(x_mb) 
          mps1_output = mps1(x_mb) 
          # Backpropagation
          loss = mse_loss(mps1_output, teacher_middle_output)
          loss.backward()
          mps1_optimizer.step()
          mps1_optimizer.zero_grad()

        # Train mps2
        else:
          student_output = LogSoftmax( student(x_mb) )
          teacher_output = LogSoftmax( teacher(x_mb) )
          # Backpropagation
          loss = student_loss_HP(student_output, teacher_output)
          loss.backward()
          mps2_optimizer.step()
          mps2_optimizer.zero_grad()


    if (epoch >= nepochs_student_HP):
      #stud_train_loss.append( round(get_acc(student, train_iterator).item(), 3) )
      stud_test_loss.append( round(get_acc(student, test_iterator).item(), 5) )
      print(stud_test_loss)
      print(stud_train_loss)

print("Student results:")
print("Epochs: ", np.arange(1, nepochs_student_HP+1).tolist())
print("Train loss: ", stud_train_loss)
print("Test loss: ", stud_test_loss)


[0.1]
[]
[0.1, 0.1]
[]
[0.1, 0.1, 0.1]
[]
[0.1, 0.1, 0.1, 0.12209]
[]
[0.1, 0.1, 0.1, 0.12209, 0.12365]
[]
[0.1, 0.1, 0.1, 0.12209, 0.12365, 0.11153]
[]
[0.1, 0.1, 0.1, 0.12209, 0.12365, 0.11153, 0.16209]
[]
[0.1, 0.1, 0.1, 0.12209, 0.12365, 0.11153, 0.16209, 0.19439]
[]
[0.1, 0.1, 0.1, 0.12209, 0.12365, 0.11153, 0.16209, 0.19439, 0.21025]
[]
[0.1, 0.1, 0.1, 0.12209, 0.12365, 0.11153, 0.16209, 0.19439, 0.21025, 0.23537]
[]
[0.1, 0.1, 0.1, 0.12209, 0.12365, 0.11153, 0.16209, 0.19439, 0.21025, 0.23537, 0.25869]
[]
[0.1, 0.1, 0.1, 0.12209, 0.12365, 0.11153, 0.16209, 0.19439, 0.21025, 0.23537, 0.25869, 0.29623]
[]
[0.1, 0.1, 0.1, 0.12209, 0.12365, 0.11153, 0.16209, 0.19439, 0.21025, 0.23537, 0.25869, 0.29623, 0.29691]
[]
[0.1, 0.1, 0.1, 0.12209, 0.12365, 0.11153, 0.16209, 0.19439, 0.21025, 0.23537, 0.25869, 0.29623, 0.29691, 0.30466]
[]
[0.1, 0.1, 0.1, 0.12209, 0.12365, 0.11153, 0.16209, 0.19439, 0.21025, 0.23537, 0.25869, 0.29623, 0.29691, 0.30466, 0.32194]
[]
[0.1, 0.1, 0.1, 0.12209, 0.1